# Convergence and performance

To optimize simulation time vs accuracy, an estimate of the acaling of the algorithm is required. For this purpose, there are two methods provided in the `Cell` and the `Layer` classes.

Below a simple example is set up to quickly run these tests. The structure here is a 300 nm layer of silicon on glass with cylindrical air holes arranged in a square lattice of periodicity 1 μm.

In [1]:
import sys
sys.path.append('../../') # Modify this line

import beams as bm
import numpy as np
from time import time
import matplotlib.pyplot as plt

p = bm.Vector2d(1, 1)

air = bm.Material()
si = bm.Material(index=3.4) 
sio2 = bm.Material(epsilon=1.5 ** 2)

hole = bm.Ellipse(r=.3, material=air) 
phc = bm.Layer(h=.3, material=si, shapes=[hole], resolution=500)

## Linear algebra

The size of the linear algebra problems is determined by the number of modes $N$. The accuracy and performance of `Layer.compute_eigs` and `Cell.linsolve` methods can be obtained using `Cell.convergence`.

#### Parameters
- `N_max`: The maximum number of modes to compute upto. The method will run the simulation for every odd integer from 1 to `N_max`.
- `**kwargs`: The frequency and angles that are passed to the `Cell.R_T()` method.

In [2]:
bm.Layer_Numba()

In Numba Layer


Layer with h = 0.0,
shapes = [],
background = <beams.materials.Material object at 0x7fa5045c9ac0>,
resolution = None
No FFT matrices stored
No eigenvalue solutions stored

In [ ]:
inc = bm.Layer()
sub = bm.Layer(material=sio2)
cell = bm.Cell(period=p, N=5, layers=[inc, phc, sub])

f = 0.75
a = bm.Vector3d(np.pi / 3, np.pi / 6, np.pi / 2)

N_max = 35
(RT, tt) = cell.convergence(N_max, freq=f, angles=a)

N = (1, 1): R = 0.18102, T = 1.08101
0.095s taken per iteration.
N = (3, 3): R = 0.72762, T = 0.27238
0.06s taken per iteration.
N = (5, 5): R = 0.74656, T = 0.25344
0.109s taken per iteration.
N = (7, 7): R = 0.63095, T = 0.36905
0.354s taken per iteration.
N = (9, 9): R = 0.61906, T = 0.38094
1.236s taken per iteration.
N = (11, 11): R = 0.61294, T = 0.38706
4.404s taken per iteration.
N = (13, 13): R = 0.61003, T = 0.38997
10.815s taken per iteration.
N = (15, 15): R = 0.60809, T = 0.39191
26.083s taken per iteration.
N = (17, 17): R = 0.60675, T = 0.39325
53.591s taken per iteration.
N = (19, 19): R = 0.60581, T = 0.39419
103.782s taken per iteration.


In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=[12.8, 4.8])
ni = np.arange(1, N_max + 1, 2) ** 2

ax[0].semilogy(ni[1:], np.abs(np.diff(RT[0])), linewidth=2)
ax[0].set_xlabel('Number of modes')
ax[0].set_ylabel('Error')

ax[1].semilogy(ni, tt, linewidth=2)
ax[1].set_xlabel('Number of modes')
ax[1].set_ylabel('Time (s)')
plt.show()

## FFT

The scaling of the FFT can be obtained through the `Layer.fft_convergence` method which returns a tuple of the time taken for each computation and the maximum difference in the norms of the FFT matrices compared to the previous computation.

#### Parameters
- `max_res`: The maximum resolution to compute the FFT for.
- `n_res`: The number of steps in the resolution from the minimum i.e. $2N-1$ to `max_res`.
- `N`: Defines the minimum resolution and truncates the central $2N-1$ orders from the FFT solution.
- `period`: The period of the lattice. Increasing the period will also increase size of grid if resolution is constant.
- `n_iter=3`: The number of computations over which the time is averaged.

In [ ]:
m_res = 1e4
n = 50
N = 9
(err, T) = phc.fft_convergence(m_res, n, N, p)

Since the values are stored, the scaling and accuracy can be quickly visualized.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=[12.8, 4.8])
ri = np.linspace(2 * N - 1, m_res, n)

ax[0].semilogy(ri, err, linewidth=2)
ax[0].set_xlabel('Resolution (px/μm)')
ax[0].set_ylabel('Error')

ax[1].plot(ri, T, linewidth=2)
ax[1].set_xlabel('Resolution (px/μm)')
ax[1].set_ylabel('Time (s)')
plt.show()